In [1]:
import rs_classes.async_request_client as rs

token = ""
base_url = "https://us.app.rossum.ai/api"
annotation_list = []
client = rs.AsyncRequestClient(token, base_url)

In [ ]:
import rs_classes.hooks as hs
import rs_functions.fetch_annotations_meta as fetch_annotations_meta
import rs_functions.fetch_annotation_content as fetch_annotation_content
import rs_functions.mongo_viewer_functions as mvf

annotations_collection = await fetch_annotations_meta.get_annotation_meta(client, annotation_list)
await fetch_annotation_content.get_annotation_content(client, annotations_collection)
hooks = await mvf.collect_hooks_per_annotation(client, annotations_collection)
